In [1]:
import pandas as pd
from pathlib import Path
import uuid

In [2]:
fincausal_task1_train_path = Path('/media/sarthak/HDD/data_science/fnp_resources/data/task1/iteration_2/train.csv')
ade_all_data_path = Path('/media/sarthak/HDD/data_science/fnp_resources/data/extra/ade.csv')

In [3]:
ade_df = pd.read_csv(ade_all_data_path)
ade_df.head()

,sentences,causal
0,At 1 year of age cardiovascular and renal func...,0
1,Chlorambucil-induced chromosome damage to huma...,1
2,"Gingival hemorrhage, myelodysplastic syndromes...",0
3,The development of safer and more effective me...,0
4,Molluscum contagiosum is a common viral skin i...,0


In [4]:
ade_df.shape

(20896, 2)

In [5]:
fincausal_task1_train_df = pd.read_csv(fincausal_task1_train_path)
fincausal_task1_train_df.head()

,Index,Text,Gold,unique_id
0,81.00009,Look at which clients are delivering increasi...,0,849c40bc-8fce-433b-8dc8-6af727480679
1,357.00082,From foundation modules that introduce asset ...,0,694a6824-613c-4978-9e17-a45b77e2b485
2,471.00003,The law firm Kessler Topaz Meltzer & Check LL...,0,661e6bf7-ffbf-4581-acff-8c246ecac51a
3,166.00001,Upcoming events Learn about the requirements...,0,48a694c5-9372-48d2-b2cb-df9461c713fe
4,48.00012,"The Midwest, where Buffett is from, has a his...",0,a363e188-0393-433b-a86c-94d268eb473c


In [6]:
fincausal_task1_train_df.shape

(15575, 4)

In [7]:
# what's the distribution of label?
causal_distribution = fincausal_task1_train_df.Gold.value_counts() / len(fincausal_task1_train_df)
causal_distribution

0    0.927897
1    0.072103
Name: Gold, dtype: float64

In [8]:
# how many causal can I take from semeval?
ade_noncausal = ade_df.loc[ade_df.causal == 0]
ade_causal = ade_df.loc[ade_df.causal == 1]

ade_causal_resampled = ade_causal.sample(n=int((causal_distribution[1]*100*len(ade_noncausal))/(100-(causal_distribution[1]*100))))

ade_resampled = pd.concat([ade_noncausal, ade_causal_resampled])
ade_resampled.causal.value_counts() / len(ade_resampled)

0    0.927942
1    0.072058
Name: causal, dtype: float64

In [9]:
# combine both the datasets
ade_resampled_formatted_df = pd.DataFrame({
    'Index': None,
    'Text': ade_resampled.sentences,
    'Gold': ade_resampled.causal,
    'unique_id': [str(uuid.uuid4()) for _ in range(len(ade_resampled))]
})
ade_resampled_formatted_df.head()

,Index,Text,Gold,unique_id
0,None,At 1 year of age cardiovascular and renal func...,0,cae7f595-c605-4b0c-996c-728a10c4ad96
2,None,"Gingival hemorrhage, myelodysplastic syndromes...",0,0d56be31-253f-49ec-a4ac-ca6b0c778e84
3,None,The development of safer and more effective me...,0,31c98f67-7267-4355-b7a6-7e71d7c9d673
4,None,Molluscum contagiosum is a common viral skin i...,0,37c0fd4a-96bd-4191-ba2f-c0f58c94d72b
5,None,Diagnosis requires CT scan and direct aspirati...,0,71e48b87-87f9-4d3c-a1b6-df1debb3e792


In [10]:
ade_resampled_formatted_df.Gold.value_counts() / len(ade_resampled_formatted_df)

0    0.927942
1    0.072058
Name: Gold, dtype: float64

In [11]:
fincausal_task1_train_combined_ade = pd.concat([fincausal_task1_train_df, ade_resampled_formatted_df])
fincausal_task1_train_combined_ade.shape

(33491, 4)

In [12]:
fincausal_task1_train_combined_ade.Gold.value_counts() / len(fincausal_task1_train_combined_ade)

0    0.927921
1    0.072079
Name: Gold, dtype: float64

In [13]:
fincausal_task1_train_combined_ade.to_csv('/media/sarthak/HDD/data_science/fnp_resources/data/task1/iteration_2_with_ade/train_with_ade.csv', index=False)